# 01 Scraping mit Playwright

In [1]:
## zur Installation der verwendeten Pakete:
# !pip install playwright pandas aiohttp aiofiles os ssl certifi
# einfach die Zeile mit !pip auskommentieren und durchführen; danach Kernel neu laden
# playwright install ##in shell ausführen

### Wie sollen die Daten am Ende aussehen?
- Id: Nummer des Posts
- author: Donald Trump @realdonaldtrump
- platform: Truthsocial or X (Twitter)
- date: ganzes Datum (ohne Uhrzeit)
- day: Tag des Posts
- month: Monat des Posts
- year: Jahr des Posts
- time: Uhrzeit des Posts
- text: ganzer Text (ohne Datum, Uhrzeit, Autor und Plattform)
- image : image_path -> der Weg zu den Bildern wird lokal gespeichert.

### Wie sieht die Website aus:

- Quellcode auf Website anschauen: https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc
- Seite lädt Inhalte interaktiv mit Java-Script nach (nicht statisch)
- alle Posts sind in jeweils einzelnen Blöcken gespeichert
- im jeweiligen Block ist einmal das Bild gespeichert und zudem DAutor, Plattform, Datum, Uhrzeit und Text in einem gemeinsamen Block
- Suchmaske auf der Website implementiert
- tägliche Ergänzung neuer Posts
- Blick auf die URL: Beim Scrollen ändert sich die Seitenzahl in der URL
- zum 1.August waren es 87.640 Posts (X und Truthsocial)
- wahrscheinlich circa 5.000 Seiten

### Wahl des Tools:
- Beautifulsoup: schon älter, nur für statische Websites geeignet, braucht länger
- Selectolax: modern und deutlich schneller als Beautiful, wird außerdem seltener blockiert, allerdings ebenfalls nur für statische Seiten
- Selenium: dynamische Alternative, gute Möglichkeit
- Playwright: relativ modern, sehr schnell und effizient, für dynamische Seiten geeignet

### Wie viele Posts gibt es überhaupt?
- Auf der Webseite steht, dass es 87.656 gibt (Stand 6.8.2025)

#### Probleme und Lösungen:
- die Seite lädt dynamisch nach: Playwright verwenden für dynamische Webseiten
- dynamisches Nachladen: wie komme ich zum Ende der Posts?
- tausend doppelte und dreifache Posts: durch Seiten durchiterieren bringt leider nur doppelte Posts
- doppelte Posts: Key erstellen, mit dem abgeglichen werden kann
- dynamisches Nachladen der Seiten: statt durch Seiten iterieren lieber Scrollen!
- Seiten laden langsam nach: sleep einbauen
- durch das Herunterladen der Bilder: Programm wird sehr langsam :(, daher parallele Worker etablieren & asynchrone Methoden, statt synchron
- Programm stürzt ab, bzw findet keine nicht immer neue Posts, weil die Seiten langsam nachladen: länger warten (sleep(3)) und Posts direkt in csv speichern und nach einem Neustart bereits Gespeichertes aus dem File laden

In [ ]:
##mit Fortsetzung bei Abbruch, final?

In [ ]:
# lädt langsam und ab 1000 Posts nicht mehr neu :(
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import hashlib
import aiofiles

nest_asyncio.apply()
os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

CSV_FILE = "trump_playwright_final.csv"

# Anzahl paralleler Download-Worker
num_workers = 10

async def download_worker(queue, session):
    os.makedirs("images", exist_ok=True)  # einmalig vor der Schleife
    while True:
        item = await queue.get()
        if item is None:  # Stop-Signal
            queue.task_done()
            break

        image_url, post, filename = item #statt post_idx
        try:
            async with session.get(image_url, ssl=sslcontext) as resp:
                if resp.status == 200:
                    fpath = os.path.join("images", filename)
                    async with aiofiles.open(fpath, "wb") as f:
                        await f.write(await resp.read())
                    #with open(fpath, "wb") as f:
                        #f.write(await resp.read())
                    #posts_data[post_idx]["image_path"] = fpath
                    post["image_path"] = fpath
        except Exception as e:
            print(f"Fehler beim Download {image_url}: {e}")
        finally:
            queue.task_done()
            

            
def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        # Content-Zeilen finden
        #if len(lines) > 2 and lines[2].startswith("View"): # statt "View on"
        #    content_lines = lines[3:]
        #else:
        #    content_lines = lines[2:]
        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on..."
            start_idx = 3
        content_lines = lines[start_idx:]

    # Text sauber zusammensetzen (mit Absatz-Trennung)
    content_text = "\n".join(content_lines).strip()

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text # statt "\n".join(content_lines).strip()
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text #"\n".join(content_lines).strip()
        }
# -------- Duplikat-Key --------
def make_post_key(data, include_image=True, include_text=True):
    """
    Generiert einen eindeutigen Schlüssel für einen Post.
    - Text optional
    - Bild optional
    - Nur Posts ohne jegliche Metadaten UND ohne Text UND ohne Bild werden verworfen
    """

    # Metadaten absichern (NaN -> "", None -> "")
    author = str(data.get("author", "") or "").strip()
    platform = str(data.get("platform", "") or "").strip()
    date = str(data.get("date", "") or "").strip()
    time = str(data.get("time", "") or "").strip()
    text = str(data.get("text", "") or "").strip()
    img = data.get("image_path")

    # Wenn absolut nichts da → kein valider Key
    if not (author or platform or date or text or img):
        return None

    parts = []
    parts.extend([author, platform, date, time])

    # Text einbauen (optional)
    if include_text and text:
        # normalize whitespace → verhindert Unterschiede nur wegen \n oder Mehrfach-Leerzeichen
        text_norm = re.sub(r"\s+", " ", text)
        parts.append(text_norm)

    # Bild einbauen (optional)
    if include_image and isinstance(img, str) and img.strip():
        parts.append(os.path.basename(img.strip()))

    # Key zusammenbauen
    raw_key = "|".join(parts).strip()
    if not raw_key:
        return None

    return hashlib.md5(raw_key.encode("utf-8")).hexdigest()


async def scrape_all_dynamic(max_posts=900, max_no_new=5):
    posts_data = []
    seen_posts = set()

    # --- Fortsetzung ---
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE).fillna("") #fillna neu
        posts_data = df_existing.to_dict("records")
        seen_posts = {make_post_key(row) for row in posts_data if make_post_key(row)} #ab if neu
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")
    else:
        posts_data = []
        seen_posts = set()

    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)

        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, session)) for _ in range(num_workers)]

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                blocks = await page.query_selector_all("div.block")
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")

            new_count = 0
            for block in blocks:
                if len(posts_data) >= max_posts:
                    break

                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Basis-Post
                    post = {
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": None,
                        "image_url": None #neu
                    }

                    # Bild-Download vorbereiten
                    img_src = None #neu
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                post["image_url"] = src #neu
                                img_src = src #neu
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                post["image_path"] = filename #neu
                                await queue.put((src, post, filename))
                    except Exception as e:
                        print(f"Bildfehler: {e}")

                    post["image_url"] = img_src #neu
                    
                    # Schlüssel generieren
                    key = make_post_key(post, include_text=True, include_image=True)
                    if not key or key in seen_posts:
                        continue
                    seen_posts.add(key)
                    
                    posts_data.append(post)
                    new_count += 1

                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            print(f"Neu hinzugekommen: {new_count} Posts")

            # Scrollen und no_new_rounds prüfen
            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen nur, wenn max_posts noch nicht erreicht
            if len(posts_data) < max_posts:
                last_height = await page.evaluate("document.body.scrollHeight")
                await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
                await asyncio.sleep(3)
                new_height = await page.evaluate("document.body.scrollHeight")
                if new_height == last_height:
                    print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                    break
            else:
                print(f"Maximale Anzahl {max_posts} erreicht.")
                break

        # Queue abwarten
        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)

        await browser.close()

    # IDs vergeben und CSV speichern
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    fff = pd.DataFrame(posts_data)
    cols = ["id", "author", "platform", "date", "time", "day", "month", "year", "text", "image_path"]
    fff = fff[cols]
    fff.to_csv(CSV_FILE, index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(fff)} Posts.")


# --- Analyse: Wie viele Posts enthalten Text ---
    total = len(fff)
    no_text = fff["text"].isna().sum()
    with_text = total - no_text
    print("===================================")
    print(f"Gesamt:     {total}")
    print(f"Ohne Text:  {no_text} ({no_text/total:.1%})")
    print(f"Mit Text:   {with_text} ({with_text/total:.1%})")
    print("===================================")

# Starten
await scrape_all_dynamic(max_posts=90000)

Vorhandene Datei gefunden: trump_playwright_final.csv – Lade gespeicherte Posts...
1151 Posts bereits geladen – setze fort...
Aktuell 1151 Posts gespeichert – 53 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 0 Posts
Keine neuen Posts (1/5)
Aktuell 1151 Posts gespeichert – 103 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 0 Posts
Keine neuen Posts (2/5)
Aktuell 1151 Posts gespeichert – 151 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 0 Posts
Keine neuen Posts (3/5)
Aktuell 1151 Posts gespeichert – 201 Blöcke auf der Seite sichtbar


In [ ]:
import asyncio
import nest_asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright
import aiohttp
import os
import certifi
import ssl
import hashlib
import aiofiles

nest_asyncio.apply()
os.makedirs("images", exist_ok=True)

sslcontext = ssl.create_default_context(cafile=certifi.where())
sslcontext.check_hostname = False
sslcontext.verify_mode = ssl.CERT_NONE

CSV_FILE = "factbase_posts_clean.csv" #"trump_playwright_final.csv"

# Anzahl paralleler Download-Worker
num_workers = 10

async def download_worker(queue, session):
    os.makedirs("images", exist_ok=True)  # einmalig vor der Schleife
    while True:
        item = await queue.get()
        if item is None:  # Stop-Signal
            queue.task_done()
            break

        image_url, post, filename = item #statt post_idx
        try:
            async with session.get(image_url, ssl=sslcontext) as resp:
                if resp.status == 200:
                    fpath = os.path.join("images", filename)
                    async with aiofiles.open(fpath, "wb") as f:
                        await f.write(await resp.read())
                    post["image_path"] = fpath
        except Exception as e:
            print(f"Fehler beim Download {image_url}: {e}")
        finally:
            queue.task_done()
            

            
def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on ..."
            start_idx = 3
        content_lines = lines[start_idx:]

    # Absatzmarker durch Leerzeichen ersetzen
    content_text = " ".join(content_lines).strip()

    # Doppelte Leerzeichen normalisieren
    content_text = re.sub(r"\s{2,}", " ", content_text)

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text
        }
    
# -------- Duplikat-Key --------
def make_post_key(data, include_image=False, include_text=True):
    """
    Generiert einen eindeutigen Schlüssel für einen Post.
    - Text optional
    - Bild optional
    - Nur Posts ohne jegliche Metadaten UND ohne Text UND ohne Bild werden verworfen
    """
    # --- Gültigkeit prüfen ---
    # -> wenn keine Metadaten vorhanden, kein Key
    # Metadaten prüfen
    #author = str(data.get("author", "")).strip()
    platform = str(data.get("platform", "")).strip()
    date = str(data.get("date", "")).strip()
    time = str(data.get("time", "")).strip()
    #text = str(data.get("text", "")).strip()
    img = data.get("image_path")
    
     # Wenn gar nichts da ist → kein valider Key
    if not (platform or date or time or img): #Text kann auch leer sein, daher weglassen
        return None
    
    parts = []
    # Basis-Metadaten immer einbauen
    parts.extend([platform, date, time])

    # Text (optional)
    if include_text:
        text_val = str(data.get("text", "")).strip().lower()
        text_norm = re.sub(r"\s+", " ", text_val)
        parts.append(text_norm)
    
    # Bild (optional)
    if include_image and isinstance(img, str) and img.strip():
        parts.append(os.path.basename(img.strip()))

    raw_key = "|".join(parts).strip()
    if not raw_key:
        return None

    return hashlib.md5(raw_key.encode("utf-8")).hexdigest()


async def scrape_all_dynamic(max_posts=90000, max_no_new=5):
    posts_data = []
    seen_posts = set()

    # --- Fortsetzung ---
    if os.path.exists(CSV_FILE):
        print(f"Vorhandene Datei gefunden: {CSV_FILE} – Lade gespeicherte Posts...")
        df_existing = pd.read_csv(CSV_FILE)
        posts_data = df_existing.to_dict("records")
        seen_posts = {make_post_key(row) for row in posts_data}
        print(f"{len(posts_data)} Posts bereits geladen – setze fort...")

    no_new_rounds = 0

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        page.set_default_timeout(60000)

        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")
        await asyncio.sleep(2)

        queue = asyncio.Queue()
        workers = [asyncio.create_task(download_worker(queue, session)) for _ in range(num_workers)]

        while True:
            try:
                await page.wait_for_selector("div.block", timeout=30000)
                blocks = await page.query_selector_all("div.block")
            except:
                print("Keine weiteren Posts, breche ab.")
                break

            print(f"Aktuell {len(posts_data)} Posts gespeichert – {len(blocks)} Blöcke auf der Seite sichtbar")

            new_count = 0
            for block in blocks:
                if len(posts_data) >= max_posts:
                    break

                try:
                    full_text = await block.inner_text()
                    data = extract_metadata_text(full_text)

                    # Basis-Post
                    post = {
                        "author": data["author"],
                        "platform": data["platform"],
                        "date": data["date"],
                        "time": data["time"],
                        "day": data["day"],
                        "month": data["month"],
                        "year": data["year"],
                        "text": data["text"],
                        "image_path": None,
                        "image_url": None #neu
                    }

                    # Bild-Download vorbereiten
                    img_src = None #neu
                    try:
                        img_el = await block.query_selector("img")
                        if img_el:
                            src = await img_el.get_attribute("src")
                            if src and re.search(r"\.jpe?g", src, re.IGNORECASE):
                                img_src = src #neu
                                filename = f"{hashlib.md5(src.encode()).hexdigest()}.jpg"
                                await queue.put((src, post, filename))
                    except:
                        pass

                    post["image_url"] = img_src #neu
                    
                    # Schlüssel generieren
                    key = make_post_key(post, include_image=False, include_text=True)
                    if not key or key in seen_posts:
                        continue
                    seen_posts.add(key)
                    
                    posts_data.append(post)
                    new_count += 1

                except Exception as e:
                    print(f"Fehler bei Post: {e}")

            print(f"Neu hinzugekommen: {new_count} Posts")

            # Scrollen und no_new_rounds prüfen
            if new_count == 0:
                no_new_rounds += 1
                print(f"Keine neuen Posts ({no_new_rounds}/{max_no_new})")
                if no_new_rounds >= max_no_new:
                    break
            else:
                no_new_rounds = 0

            # Scrollen nur, wenn max_posts noch nicht erreicht
            if len(posts_data) < max_posts:
                last_height = await page.evaluate("document.body.scrollHeight")
                await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
                await asyncio.sleep(3)
                new_height = await page.evaluate("document.body.scrollHeight")
                if new_height == last_height:
                    print(f"Scrollen brachte nichts Neues ({no_new_rounds}/{max_no_new})")
                    break
            else:
                print(f"Maximale Anzahl {max_posts} erreicht.")
                break

        # Queue abwarten
        await queue.join()
        for _ in range(num_workers):
            await queue.put(None)
        await asyncio.gather(*workers)

        await browser.close()

    # IDs vergeben und CSV speichern
    for idx, post in enumerate(posts_data, start=1):
        post["id"] = idx

    fff = pd.DataFrame(posts_data)
    cols = ["id", "author", "platform", "date", "time", "day", "month", "year", "text", "image_path"]
    fff = fff[cols]
    fff.to_csv(CSV_FILE, index=False, encoding="utf-8")
    print(f"Scraping abgeschlossen. Gesamt: {len(fff)} Posts.")


# --- Analyse: Wie viele Posts enthalten Text ---
    total = len(fff)
    no_text = fff["text"].isna().sum()
    with_text = total - no_text
    print("===================================")
    print(f"Gesamt:     {total}")
    print(f"Ohne Text:  {no_text} ({no_text/total:.1%})")
    print(f"Mit Text:   {with_text} ({with_text/total:.1%})")
    print("===================================")

# Starten
await scrape_all_dynamic(max_posts=90000)

Vorhandene Datei gefunden: factbase_posts_clean.csv – Lade gespeicherte Posts...
6381 Posts bereits geladen – setze fort...
Aktuell 6381 Posts gespeichert – 55 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 5 Posts
Aktuell 6386 Posts gespeichert – 105 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 4 Posts
Aktuell 6390 Posts gespeichert – 151 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 8 Posts
Aktuell 6398 Posts gespeichert – 201 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 9 Posts
Aktuell 6407 Posts gespeichert – 251 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 6 Posts
Aktuell 6413 Posts gespeichert – 301 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 7 Posts
Aktuell 6420 Posts gespeichert – 351 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 8 Posts
Aktuell 6428 Posts gespeichert – 401 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 4 Posts
Aktuell 6432 Posts gespeichert – 451 Blöcke auf der Seite sichtbar
Neu hinzugekommen: 5 Posts
Aktuell 6437 Posts gespeichert 

In [6]:
import pandas as pd
ppp = pd.read_csv("factbase_posts_clean.csv")
print(ppp.tail(10).T)

                                                         5846  \
id                                                       5847   
author                          Donald Trump @realDonaldTrump   
platform                                         Truth Social   
date                                               2024-11-03   
time                                                    01:25   
day                                                       3.0   
month                                                November   
year                                                   2024.0   
text        If Kamala wins, you are 3 days away from the s...   
image_path        images/c686ddbd8c368873f7932d52eb10d5e1.jpg   

                                                         5847  \
id                                                       5848   
author                          Donald Trump @realDonaldTrump   
platform                                                  NaN   
date                    

In [7]:
ppp.shape

(5856, 10)

In [3]:
import pandas as pd
fff = pd.read_csv("trump_playwright_final.csv")
print(fff.tail(10).T)

                                                  26775  \
id                                                26776   
author                    Donald Trump @realDonaldTrump   
platform                                   Truth Social   
date                                         2025-02-23   
time                                              12:20   
day                                                23.0   
month                                          February   
year                                             2025.0   
text                                                NaN   
image_path  images/f069d362f328857b3dfc3009e9d2e370.jpg   

                                                  26776  \
id                                                26777   
author                    Donald Trump @realDonaldTrump   
platform                                   Truth Social   
date                                         2025-02-22   
time                                              22:41

In [4]:
fff.text[10]

'https://hannity.com/media-room/out-here-in-these-streets-trump-to-patrol-d-c-with-police-military-as-part-of-crime-crackdown-report/'

In [5]:
fff.shape

(26785, 10)

In [51]:
# Wie viele leere Werten gibt es?
print(fff['time'].isna().sum(), "NaN-Werte")
print((fff['time'] == "").sum(), "leere Strings")
#print(fff['time'].apply(lambda x: repr(str(x))).value_counts().head(20))

1 NaN-Werte
0 leere Strings


In [52]:
# Wie viele Posts enthalten keine Angabe zur Zeit?
count_without_time = fff["time"].isna().sum()
print(f"Anzahl der Posts ohne Zeitangabe: {count_without_time}")

Anzahl der Posts ohne Zeitangabe: 1


In [60]:
fff_single = fff.drop_duplicates(subset=['time','date', 'image_path'], keep="first")
print(f"Vorher: {len(fff)} Zeilen, danach: {len(fff_single)} Zeilen ohne Duplikate.")

Vorher: 5928 Zeilen, danach: 5928 Zeilen ohne Duplikate.


In [8]:
ppp_single = ppp.drop_duplicates(subset=['time','date', 'image_path'], keep="first")
print(f"Vorher: {len(ppp)} Zeilen, danach: {len(ppp_single)} Zeilen ohne Duplikate.")

Vorher: 5856 Zeilen, danach: 5179 Zeilen ohne Duplikate.


In [54]:
count_without_text = fff["id"].isna().sum()
print(f"Anzahl der Posts ohne Text: {count_without_text}")

Anzahl der Posts ohne Text: 0


In [55]:
# Bereinigung der Nan-Werte
fff['text'] = fff['text'].fillna("").str.strip()

In [56]:
post_452 = fff[fff['time'] == '04:52']
print(post_452)

Empty DataFrame
Columns: [id, author, platform, date, time, day, month, year, text, image_path]
Index: []


In [57]:
import re

# --- Normalisierung wie besprochen ---
def normalize_text(x):
    if pd.isna(x):
        return "<EMPTY>"
    txt = str(x).strip().lower()
    if txt == "":
        return "<EMPTY>"
    # Links dürfen bleiben
    return txt

fff["text_norm"]   = fff["text"].apply(normalize_text)
fff["author_norm"] = fff["author"].fillna("").str.strip().str.lower()
fff["img_norm"]    = fff["image_path"].fillna("").str.strip().str.lower()
fff.loc[fff["img_norm"]=="", "img_norm"] = "<NO_IMAGE>"

mask_empty_text = fff["text_norm"] == "<EMPTY>"
mask_no_image   = fff["img_norm"]  == "<NO_IMAGE>"

# --- Grundmengen (Zeilenebene) ---
exact_rows       = fff.duplicated(["author_norm","platform","date","time","text_norm","img_norm"], keep=False)

text_equal_rows  = text_equal_rows = fff.duplicated(["text_norm"], keep=False)
 #fff.duplicated(["text_norm"], keep=False)  & ~mask_empty_text
image_equal_rows = fff.duplicated(["img_norm"],  keep=False)  & ~mask_no_image

# --- Disjunkte Aufteilung (keine Überschneidungen) ---
mixed_rows       = (text_equal_rows & image_equal_rows) & ~exact_rows
text_only_rows   = (text_equal_rows & ~image_equal_rows) & ~exact_rows
image_only_rows  = (image_equal_rows & ~text_equal_rows) & ~exact_rows

# exact_rows bleibt separat (vollständig identische Zeilen)

# --- Prüfen, dass alles disjunkt ist ---
assert not any([
    (text_only_rows & image_only_rows).any(),
    (text_only_rows & mixed_rows).any(),
    (image_only_rows & mixed_rows).any(),
    (exact_rows & (text_only_rows | image_only_rows | mixed_rows)).any()
])

# --- Eindeutig einzigartige Zeilen ---
unique_rows = ~(exact_rows | text_only_rows | image_only_rows | mixed_rows)

# --- Kennzahlen ---
print("== Übersicht ==")
print(f"Gleicher Text (ALLE, inkl. exakter):  {text_equal_rows.sum()}")
print(f"Gleiche Bilder (ALLE, inkl. exakter): {image_equal_rows.sum()}")
print(f"Exakte Duplikat-Zeilen:               {exact_rows.sum()}")
print("Exakte Duplikate mit leerem Text:", (exact_rows & mask_empty_text).sum())
print("Exakte Duplikate mit echtem Text:", (exact_rows & ~mask_empty_text).sum())
print(f"Nur Text-Duplikate (nicht exakt):     {text_only_rows.sum()}")
print(f"Nur Bild-Duplikate (nicht exakt):     {image_only_rows.sum()}")
print(f"Text+Bild dupl. (nicht exakt):        {mixed_rows.sum()}")
print(f"Echte einzigartige Zeilen:            {unique_rows.sum()}")
print(f"Gesamt:                               {len(fff)}")

# --- Exporte ---
def save_groups(df, mask, group_cols, filename):
    subset = df[mask].copy()
    if subset.empty:
        print(f"Keine Gruppen für {filename}")
        return
    subset["dupe_group"] = subset.groupby(group_cols).ngroup()
    subset.sort_values(group_cols + ["date","time"], inplace=True)
    subset.to_csv(filename, index=False, encoding="utf-8")
    print(f"Gespeichert: {filename} ({len(subset)} Zeilen)")

# Gruppenreports:
# 1) „Gleicher Text (ALLE)“ – das ist die Menge, die du intuitiv meinst
save_groups(fff, text_equal_rows, ["text_norm"], "groups_text_all.csv")

# 2) „Nur Text-Duplikate, nicht exakt“
save_groups(fff, text_only_rows, ["text_norm"], "groups_text_only.csv")

# 3) Exakt
save_groups(fff, exact_rows, ["author_norm","platform","date","time","text_norm","img_norm"], "groups_exact.csv")

# 4) Bilder
save_groups(fff, image_equal_rows, ["img_norm"], "groups_images_all.csv")
save_groups(fff, image_only_rows, ["img_norm"], "groups_images_only.csv")

# 5) Mixed (gleicher Text & gleiches Bild, aber nicht komplett identisch)
save_groups(fff, mixed_rows, ["text_norm","img_norm"], "groups_mixed.csv")

# Echte Uniques
fff[unique_rows].to_csv("unique_posts.csv", index=False, encoding="utf-8")
print("Gespeichert: unique_posts.csv")


== Übersicht ==
Gleicher Text (ALLE, inkl. exakter):  26059
Gleiche Bilder (ALLE, inkl. exakter): 25935
Exakte Duplikat-Zeilen:               25935
Exakte Duplikate mit leerem Text: 4591
Exakte Duplikate mit echtem Text: 21344
Nur Text-Duplikate (nicht exakt):     124
Nur Bild-Duplikate (nicht exakt):     0
Text+Bild dupl. (nicht exakt):        0
Echte einzigartige Zeilen:            726
Gesamt:                               26785
Gespeichert: groups_text_all.csv (26059 Zeilen)
Gespeichert: groups_text_only.csv (124 Zeilen)
Gespeichert: groups_exact.csv (25935 Zeilen)
Gespeichert: groups_images_all.csv (25935 Zeilen)
Keine Gruppen für groups_images_only.csv
Keine Gruppen für groups_mixed.csv
Gespeichert: unique_posts.csv


In [65]:
import re
from datetime import datetime

def extract_metadata_text(text: str):
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    author_name = ""
    handle = ""
    platform = ""
    date_str = ""
    time_str = ""
    content_lines = []

    if len(lines) >= 2:
        author_name = lines[0].strip()
        match = re.search(
            r"(@[\w]+)\s*[•\-]\s*(.*?)\s*[•\-]\s*([A-Za-z]+ \d{1,2}, \d{4})\s*@\s*(\d{1,2}:\d{2} [AP]M)",
            lines[1]
        )
        if match:
            handle = match.group(1).strip()
            platform = match.group(2).strip()
            date_str = match.group(3).strip()
            time_str = match.group(4).strip()

        start_idx = 2
        if len(lines) > 2 and lines[2].startswith("View"):  # z.B. "View on ..."
            start_idx = 3
        content_lines = lines[start_idx:]

    # Absatzmarker durch Leerzeichen ersetzen
    content_text = " ".join(content_lines).strip()

    # Doppelte Leerzeichen normalisieren
    content_text = re.sub(r"\s{2,}", " ", content_text)

    try:
        dt = datetime.strptime(f"{date_str} {time_str}", "%B %d, %Y %I:%M %p")
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": dt.strftime("%Y-%m-%d"),
            "time": dt.strftime("%H:%M"),
            "year": int(dt.year),
            "month": dt.strftime("%B"),
            "day": int(dt.day),
            "text": content_text
        }
    except Exception:
        return {
            "author": f"{author_name} {handle}".strip(),
            "platform": platform,
            "date": date_str,
            "time": time_str,
            "year": "",
            "month": "",
            "day": "",
            "text": content_text
        }

# --- Testfunktion ---
def test_extract_metadata():
    sample_post = """
    Donald Trump
    @realDonaldTrump • Twitter • January 6, 2021 @ 3:45 PM
    View on Twitter
    This is a test post
    with multiple lines
    and even more text.
    
    As obviously, this is also part of the text.
    
    And what about this? @you
    Look here!
    Views: 123,456
    """

    result = extract_metadata_text(sample_post)
    print("Autor:", result["author"])
    print("Plattform:", result["platform"])
    print("Datum:", result["date"])
    print("Zeit:", result["time"])
    print("Text:", result["text"])

# Testlauf
if __name__ == "__main__":
    test_extract_metadata()


Autor: Donald Trump @realDonaldTrump
Plattform: Twitter
Datum: 2021-01-06
Zeit: 15:45
Text: This is a test post with multiple lines and even more text. As obviously, this is also part of the text. And what about this? @you Look here! Views: 123,456


In [ ]:
import os

async def scrape_all_dynamic(max_posts=100000, max_no_new=5, output_file="factbase_posts_clean.csv"):
    posts_data = []
    seen_ids = set()

    # 🔹 Vorhandene CSV einlesen, falls vorhanden
    if os.path.exists(output_file):
        df_existing = pd.read_csv(output_file)
        print(f"{len(df_existing)} Posts bereits in {output_file} gefunden.")
        for _, row in df_existing.iterrows():
            key = f"{row['author']}|{row['date']}|{row['time']}|{row['text']}"
            pid = hashlib.md5(key.encode("utf-8")).hexdigest()
            seen_ids.add(pid)
        posts_data = df_existing.to_dict("records")

    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://rollcall.com/factbase-twitter/?platform=all&sort=date&sort_order=desc")

        no_new_rounds = 0
        while True:
            try:
                await page.wait_for_selector("div.block", timeout=15000)
            except:
                print("Timeout beim Warten auf Posts – Abbruch.")
                break

            blocks = await page.query_selector_all("div.block")
            print(f"Aktuell {len(blocks)} Posts im DOM")

            new_in_round = 0
            for block in blocks:
                text = (await block.inner_text()).strip()
                data = extract_metadata_text(text)
                pid = make_post_id(data)

                if pid not in seen_ids:
                    seen_ids.add(pid)
                    posts_data.append(data)
                    new_in_round += 1

            print(f"Neue eindeutige Posts in dieser Runde: {new_in_round}")

            if new_in_round == 0:
                no_new_rounds += 1
                if no_new_rounds >= max_no_new:
                    print("Mehrfach keine neuen Posts gefunden – beende Scraping.")
                    break
            else:
                no_new_rounds = 0

            print(f"Gesamt bisher gespeichert: {len(posts_data)}")

            if len(posts_data) >= max_posts:
                print("Maximale Anzahl Posts erreicht.")
                break

            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(2000)

        await browser.close()

    # 🔹 Kombinierte Daten abspeichern
    df = pd.DataFrame(posts_data)
    df.to_csv(output_file, index=False)
    print(f"Scraping abgeschlossen. Gesamt: {len(df)} eindeutige Posts gespeichert in {output_file}.")
